In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request
import time

In [2]:
years = np.arange(2016,2019,1).tolist()

for year in years:
    web_name = 'https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html'
    website = request.urlopen(web_name).read()
    soup = BeautifulSoup(website,'lxml')

    table = soup.find('table', attrs={'class': 'sortable', 'id': 'per_game_stats'})
    table_headers = [header.text for header in table.find('thead').find_all('th')]
    table_rows = table.find_all('tr')

    final_data = []
    # For each table row,
    for tr in table_rows:
        # Make a list of the table data tags for this row
        td = tr.find_all('td')
        # Extract just the cell data and make a list
        row = [tr.text for tr in td]
        # Append the extracted data
        final_data.append(row)


    df = pd.DataFrame(final_data[1:], columns=table_headers[1:])
    df.to_csv('../data/nba_stats_'+str(year)+'.csv')
    
    time.sleep(1)

In [4]:
table = soup.find('table', attrs={'class': 'sortable', 'id': 'per_game_stats'})

table_headers = [header.text for header in table.find('thead').find_all('th')]
table_rows = table.find_all('tr')

In [5]:
final_data = []
# For each table row,
for tr in table_rows:
    # Make a list of the table data tags for this row
    td = tr.find_all('td')
    # Extract just the cell data and make a list
    row = [tr.text for tr in td]
    # Append the extracted data
    final_data.append(row)

In [12]:
df = pd.DataFrame(final_data[1:], columns=table_headers[1:])
df.to_csv('../data/nba_stats_'+str(year)+'.csv')

In [9]:
df.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,.357,...,.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,.222,...,.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,.345,...,.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,.595,...,.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,.576,...,.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
